In [5]:
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
from pyproj import Proj, transform
import math

In [2]:
# Enable GDAL exceptions
gdal.UseExceptions()

# Load DEM data
# dem_file = 'C:/Users/leeyj/Downloads/Test_DEM.tif'
dem_file = "C:/Users/leeyj/OneDrive - 인하대학교/School/과제/[국방 수직이착륙기 특화연구센터(VTD-13)]/Test_DEM.tif"
dataset = gdal.Open(dem_file)
band = dataset.GetRasterBand(1)
elevation = band.ReadAsArray()

# Get geotransform info
geotransform = dataset.GetGeoTransform()

# Initialize an empty list to store the data
matrix = []

# Calculate longitude, latitude, and elevation for each pixel
for y in range(elevation.shape[0]):
    row = []
    for x in range(elevation.shape[1]):
        longitude = geotransform[0] + x*geotransform[1] + y*geotransform[2]
        latitude = geotransform[3] + x*geotransform[4] + y*geotransform[5]
        elev = elevation[y, x]
        row.append((elev, longitude, latitude))
    matrix.append(row)

# Convert the list to a NumPy array for more efficient operations and handling
matrix_np = np.array(matrix)

# Now `matrix_np` contains the elevation, longitude, and latitude for each pixel
# matrix_np.shape

In [3]:
# Since the state was reset and I don't actually have access to the file system to load a .tif file,
# the previous code snippet will serve as a basis for demonstrating how to flip the matrix.
# I'll recreate a simplified version of the `matrix_np` based on the code you mentioned,
# and then apply the vertical flipping operation to it.

# Simulating the creation of a matrix based on the provided code structure.
# Let's assume a 5x5 matrix for simplicity.
# rows, cols = 5, 5
# geotransform = [0, 1, 0, 0, 0, -1]  # Simplified geotransform for demonstration
# elevation = np.arange(rows * cols).reshape(rows, cols)  # Dummy elevation data

# matrix = []

# for y in range(elevation.shape[0]):
#     row = []
#     for x in range(elevation.shape[1]):
#         longitude = geotransform[0] + x*geotransform[1] + y*geotransform[2]
#         latitude = geotransform[3] + x*geotransform[4] + y*geotransform[5]
#         elev = elevation[y, x]
#         row.append((elev, longitude, latitude))
#     matrix.append(row)

# matrix_np = np.array(matrix)

# Flip the matrix in the up/down direction to change the indexing scheme
flipped_matrix_np = np.flipud(matrix_np)

In [4]:
# Let's verify the transformation by checking the new positions of the first and last elements
print(matrix_np[0, 0], flipped_matrix_np[3600, 0])
print(matrix_np[-1, -1], flipped_matrix_np[0, -1])

print(matrix_np[0, 0])

print(flipped_matrix_np[0,0])

[869.         126.99986111  39.00013889] [869.         126.99986111  39.00013889]
[338.         127.99986111  38.00013889] [338.         127.99986111  38.00013889]
[869.         126.99986111  39.00013889]
[ 94.         126.99986111  38.00013889]


In [ ]:
# Assuming `elevation` contains your elevation data as a 2D NumPy array
plt.figure(figsize=(10, 10))
plt.imshow(elevation, cmap='terrain')
plt.colorbar(label='Elevation (m)')
plt.title('DEM Elevation Map')
plt.xlabel('Longitude Index')
plt.ylabel('Latitude Index')
plt.show()

In [ ]:
# Assuming flipped_matrix contains the flipped data with tuples in the form of (elevation, longitude, latitude)
# First, extract just the elevation data for visualization
elevation_data_flipped = np.array([[cell[0] for cell in row] for row in flipped_matrix_np])

# Plotting the elevation data
plt.figure(figsize=(10, 10))
plt.imshow(elevation_data_flipped, cmap='terrain')
plt.colorbar(label='Elevation (m)')
plt.title('Flipped DEM Elevation Map')
plt.show()

In [ ]:
# Assuming elevation_data_flipped is your 2D array of elevation data
# Plotting the elevation data with adjusted origin and extent
plt.figure(figsize=(10, 10))
# Set the extent so that the plot matches your data's geographic extent.
# The format is [left, right, bottom, top].
# Here, we assume your data covers a grid of 3601x3601 starting from bottom-left as [0,0]
plt.imshow(elevation_data_flipped, cmap='terrain', origin='lower', extent=(0.0, 3600.0, 0.0, 3600.00))

plt.colorbar(label='Elevation (m)')
plt.xlabel('Index X')
plt.ylabel('Index Y')
plt.title('Flipped DEM Elevation Map with Corrected Indices')
plt.show()

In [ ]:
# return Longitude, Latitude, Altitude of choen user input index 
def get_data_at_index(matrix, lon_index, lat_index):
    """
    Retrieve elevation, longitude, and latitude data for a specific pixel in the matrix.
    
    Parameters:
    - matrix: The data matrix containing tuples of (elevation, longitude, latitude).
    - lon_index: The longitude index of the pixel.
    - lat_index: The latitude index of the pixel.
    
    Returns:
    - A tuple containing the elevation, longitude, and latitude of the specified pixel, or None if indices are out of bounds.
    """
    # Check if the provided indices are within the bounds of the matrix
    if lat_index >= 0 and lat_index < len(matrix) and lon_index >= 0 and lon_index < len(matrix[0]):
        return matrix[lat_index][lon_index]
    else:
        return None

In [ ]:
# Call function
lon_index = 0
lat_index = 0
data_at_index = get_data_at_index(matrix_np, lon_index, lat_index)

if data_at_index is not None:
    print(f"Elevation: {data_at_index[0]}, Longitude: {data_at_index[1]}, Latitude: {data_at_index[2]}")
else:
    print("Invalid indices provided.")

In [ ]:
# Call function
lon_index = 3600
lat_index = 0
data_at_index = get_data_at_index(flipped_matrix_np, 3600-lon_index, 3600-lat_index)

if data_at_index is not None:
    print(f"Elevation: {data_at_index[0]}, Longitude: {data_at_index[1]}, Latitude: {data_at_index[2]}")
else:
    print("Invalid indices provided.")

In [ ]:
# return maximum index of Longitude and Latitude
# Assuming `matrix_np` is your NumPy matrix
lat_max_index, lon_max_index = matrix_np.shape[0] - 1, matrix_np.shape[1] - 1

print(f"Maximum Latitude Index: {lat_max_index}")
print(f"Maximum Longitude Index: {lon_max_index}")

In [ ]:
# saving DEM data to CSV format
elevation_data = [[cell[0] for cell in row] for row in matrix_np]

# Assuming longitude and latitude values are evenly spaced and can be derived from the indices
lat_headers = [f"Lat_{i}" for i in range(matrix_np.shape[0])]
lon_headers = [f"Lon_{j}" for j in range(matrix_np.shape[1])]

# Convert elevation data to DataFrame
df = pd.DataFrame(elevation_data, columns=lon_headers)

# Optionally, if you want latitude values as the first column (as index)
df.insert(0, 'Latitude', lat_headers)

# Save to CSV
df.to_csv('elevation_matrix.csv', index=False)

In [ ]:
# Assuming matrix_np is your structured NumPy matrix with (elevation, longitude, latitude) for each cell
# First, let's extract the elevation data
elevation_data = [[cell[0] for cell in row] for row in matrix_np]

# Extract longitude and latitude values for headers
# For longitude, take the values from the first row
longitude_values = [cell[1] for cell in matrix_np[0]]  # Assuming all rows have the same longitude values across

# For latitude, take the latitude value of the last cell in each row
latlon_values = [row[-1][2] for row in matrix_np]  # Assuming all columns have the same latitude values down

# Convert elevation data into a DataFrame
df_direct_headers = pd.DataFrame(elevation_data, columns=longitude_values)

# Add latitude values as the first column of the DataFrame
df_direct_headers.insert(0, 'Latitude/Longitude', latlon_values)

# Save the DataFrame to a CSV file
# csv_path_direct = 'C:/Users/leeyj/lab_ws/source'
df_direct_headers.to_csv('NK_DEM.csv', index=False)

# Note: Replace 'path/to/your/elevation_matrix_direct_headers.csv' with your desired file path

In [ ]:
# # Path where you want to save the file
# file_path = 'flipped_matrix.txt'

# # Open the file in write mode
# with open(file_path, 'w') as file:
#     # Iterate over each row in the flipped_matrix
#     for row in flipped_matrix_np:
#         # For each cell in the row, format the data as "[longitude, latitude, elevation]"
#         formatted_row = [f"{cell[1]}, {cell[2]}, {cell[0]}" for cell in row]
#         # Join the formatted cell data with commas and write it as a new line in the file
#         file.write(','.join(formatted_row) + '\n')

# print("File has been saved successfully.")

인덱스가 바뀌어진 북한 지형 DEM 파일을 저장

In [ ]:
# Path where you want to save the CSV file
csv_file_path = 'C:/Users/leeyj/lab_ws/data/North_Korea_Flipped_DEM.csv'
# Open the file in write mode
with open(csv_file_path, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csvwriter = csv.writer(csvfile)
    
    # Iterate over each cell in the flipped_matrix and write the formatted data to the CSV file
    for row in flipped_matrix_np:
        formatted_row = [f"{cell[1]}, {cell[2]}, {cell[0]}" for cell in row]  # Format: [longitude, latitude, elevation]
        csvwriter.writerow(formatted_row)

print("CSV file has been saved successfully.")

In [ ]:
# Step 1: Load the CSV data
csv_path = 'C:/Users/leeyj/lab_ws/data/NK_DEM.csv'
df = pd.read_csv(csv_path)

df

In [ ]:
# latitudes = df['Latitude/Longitude'].values
# longitudes = df.columns[1:].astype(float)  # Convert longitude headers to float
# elevation_data = df.drop('Latitude/Longitude', axis=1).values

In [ ]:
# # Step 2: Create a new GeoTIFF file
# driver = gdal.GetDriverByName('GTiff')
# dst_filename = 'path/to/your/new_dem.tif'
# # Here we use the shape of the loaded CSV data to determine raster size
# dataset = driver.Create(dst_filename, len(longitudes), len(latitudes), 1, gdal.GDT_Float32)

# # Define the geotransform (this would need to match your original DEM's spatial reference)
# # This is a placeholder example. You'd need to use the actual values from your DEM.
# dataset.SetGeoTransform([longitudes.min(), (longitudes.max() - longitudes.min()) / len(longitudes), 0, latitudes.max(), 0, -(latitudes.max() - latitudes.min()) / len(latitudes)])

# # Set the projection (this should also match your original DEM)
# srs = osr.SpatialReference()
# srs.ImportFromEPSG(4326)  # This uses WGS84, but you should use your DEM's projection
# dataset.SetProjection(srs.ExportToWkt())

# # Step 3: Write the elevation data to the new raster
# band = dataset.GetRasterBand(1)
# band.WriteArray(elevation_data)

# # Don't forget to flush data to disk
# band.FlushCache()
# dataset = None

# # Step 4: Visualize the raster
# # This step reuses the previous visualization example
# dataset = gdal.Open(dst_filename)
# band = dataset.GetRasterBand(1)
# elevation = band.ReadAsArray()

# plt.figure(figsize=(10, 10))
# plt.imshow(elevation, cmap='terrain')
# plt.colorbar(label='Elevation (m)')
# plt.title('Reconstructed DEM Elevation Map')
# plt.show()

데이터의 Index를 변환   
좌상단 Index 0,0이 좌하단 0,0로 옮겨지며 우하단 3600,3600 Index가 우상단 3600,3600으로 이동

In [ ]:
# Assuming you have a matrix 'original_matrix' structured with 3601x3601 dimensions
# For demonstration, let's create a dummy 5x5 matrix where each element is a tuple of (altitude, longitude, latitude)
original_matrix = np.array([[(100 + i + j, 30 + 0.01*j, -90 - 0.01*i) for j in range(5)] for i in range(5)])

# Flip the matrix in the up/down direction
flipped_matrix = np.flipud(original_matrix)

# Example: Print the original and flipped matrix indices [0,0] to see the change
print("Original [0,0]:", original_matrix[0,0])
print("Flipped [0,0]:", flipped_matrix[0,0])

In [ ]:
# Initialize the WGS-84 and Lambert Conformal Conic projections
wgs84 = Proj(init='epsg:4326')  # WGS-84
lambert_conformal_conic = Proj(proj='lcc', lat_1=33, lat_2=45, lon_0=-95, ellps='WGS84')  # Example parameters

# Function to transform coordinates
def transform_coordinates(matrix_np):
    transformed_matrix = np.empty(matrix_np.shape, dtype=object)
    for y in range(matrix_np.shape[0]):
        for x in range(matrix_np.shape[1]):
            elev, lon, lat = matrix_np[y, x]
            # Transform the longitude and latitude
            lon_lcc, lat_lcc = transform(wgs84, lambert_conformal_conic, lon, lat)
            # Store the transformed coordinates along with the elevation
            transformed_matrix[y, x] = (elev, lon_lcc, lat_lcc)
    return transformed_matrix

# Assuming `matrix_np` is already loaded with your DEM data
transformed_matrix_np = transform_coordinates(matrix_np)

In [ ]:
# Constants for the Lambert Conformal Conic Projection
# These should be adjusted according to your specific needs or study area
lat_1 = 33  # First standard parallel
lat_2 = 45  # Second standard parallel
lon_0 = -95  # Central meridian
R = 6371000  # Radius of the Earth in meters (approximation)

# Convert degrees to radians
def deg_to_rad(deg):
    return deg * math.pi / 180

# Calculation of projection constants
def calculate_constants(lat_1, lat_2):
    lat_1_rad = deg_to_rad(lat_1)
    lat_2_rad = deg_to_rad(lat_2)
    
    n = math.log(math.cos(lat_1_rad) / math.cos(lat_2_rad)) / \
        math.log(math.tan(math.pi / 4 + lat_2_rad / 2) / math.tan(math.pi / 4 + lat_1_rad / 2))
    F = (math.cos(lat_1_rad) * math.pow(math.tan(math.pi / 4 + lat_1_rad / 2), n)) / n
    rho_0 = R * F * math.pow(math.tan(math.pi / 4 + deg_to_rad(90) / 2), -n)
    
    return n, F, rho_0

n, F, rho_0 = calculate_constants(lat_1, lat_2)

# Function to transform longitude and latitude to Lambert Conformal Conic
def lambert_conformal_conic_transform(elevation, lon, lat):
    lat_rad = deg_to_rad(lat)
    lon_rad = deg_to_rad(lon)
    rho = R * F * math.pow(math.tan(math.pi / 4 + lat_rad / 2), -n)
    theta = n * (lon_rad - deg_to_rad(lon_0))
    x = rho * math.sin(theta)
    y = rho_0 - rho * math.cos(theta)
    return elevation, x, y

# Apply the transformation to the entire DEM matrix
def apply_projection(matrix_np):
    transformed_matrix = np.empty(matrix_np.shape, dtype=object)
    for y in range(matrix_np.shape[0]):
        for x in range(matrix_np.shape[1]):
            elev, lon, lat = matrix_np[y, x]
            transformed_matrix[y, x] = lambert_conformal_conic_transform(elev, lon, lat)
    return transformed_matrix

# Assuming `matrix_np` is loaded with your DEM data
# transformed_matrix_np = apply_projection(matrix_np)

In [ ]:
psi = 
R = 
k = math.log(math.sin)
lamda = 
